# Cleaning up the doelpunten.csv file

In [ ]:
import pandas as pd

### Creating a dataframe

In [ ]:
df = pd.read_csv("doelpunten.csv", encoding='latin1',
names=['Seizeon', 'Speeldag', 'Datum', 'Startuur', 'Match_ID', 'Thuisploeg', 'Uitploeg',
'Minuut goal', 'Tijdstip goal', 'Goalploeg', 'Thuisstand', "Uitstand"], sep=";")

#### Adding column for minute of goal

In [ ]:
# tijdstip doelpunt in geheel getal
def convert(row):
    uur, minuut = row['Tijdstip goal'].split(":")
    start_uur, start_minuut = row['Startuur'].split(":")

    uur = int(uur)
    minuut = int(minuut)
    start_uur = int(start_uur)
    start_minuut = int(start_minuut)

    if uur < start_uur:
        absolute_minuten = (60 - start_minuut) + (uur + 1 - start_uur) * 60 + minuut
    else:
        absolute_minuten = (uur - start_uur) * 60 + minuut - start_minuut

    return absolute_minuten

In [ ]:
df['Minuut goal'] = df.apply(convert, axis=1)

In [ ]:
controle_binnen_tijd = df[df['Minuut goal'] > 120]
print(len(controle_binnen_tijd))

0


### Creating second dataframe

In [ ]:
wedstrijden_df = pd.read_csv('wedstrijden.csv', encoding='latin1',
names=['Seizoen', 'Speeldag', 'Datum', 'Startuur', 'Match_ID', 'Thuisploeg',
'Uitploeg', 'Score Thuisploeg', 'Score Uitploeg'], sep=";")

#### Merge both dataframes

In [ ]:
df_unique = df.drop_duplicates(subset=['Match_ID'], keep='last')
# wedstrijden_df_unique = wedstrijden_df.drop_duplicates(subset=['Match_ID'], keep='last')

df_unique['Match_ID'] = df_unique['Match_ID'].astype(str)
wedstrijden_df.Match_ID = wedstrijden_df.Match_ID.astype(str)

merged_df = pd.merge(df_unique, wedstrijden_df, on="Match_ID", how="inner")

<ipython-input-20-3a556ea04728>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['Match_ID'] = df_unique['Match_ID'].astype(str)


#### Vergelijken van de datums

In [ ]:
merged_df['Datums_gelijk'] = merged_df['Datum_x'] == merged_df['Datum_y']

merged_df["Datums_gelijk"].value_counts()

True     5462
False     948
Name: Datums_gelijk, dtype: int64

#### Controleren of uiteindelijke resultaat overeenkomt met doelpunten

In [ ]:
fouten = merged_df[(merged_df['Score thuisploeg_x'] != merged_df['Score thuisploeg_y']) |
                        (merged_df['Score Uitploeg_x'] != merged_df['Score Uitploeg_y'])]

if len(fouten) == 0:
    print("Geen fouten")
else:
    print(fouten.index)

In [ ]:
columns = ["club_id", "Thuisploeg", "huisstamnummer", "huisroepnaam"]
id_df = pd.read_csv("stamnummers.csv", sep=";", header=None)
id_df.columns = columns

In [ ]:
# Drop duplicate ploegen
id_df.drop(index=115, inplace=True)
id_df.drop(index=24, inplace=True)
id_df.drop(index=48, inplace=True)
id_df.drop(index=131, inplace=True)

In [ ]:
# Merge de twee dataframes on de ploeg kolom
df = pd.merge(df, id_df, on="Thuisploeg", how="left")

In [ ]:
columns = ["club_id", "Uitploeg", "uitstamnummer", "uitroepnaam"]
id_df.columns = columns
df = pd.merge(df, id_df, on="Uitploeg", how="left")

In [ ]:
columns = ["club_id", "Goalploeg", "goalstamnummer", "goalroepnaam"]
id_df.columns = columns
df = pd.merge(df, id_df, on="Goalploeg", how="left")

In [ ]:
df.info()

In [ ]:

df = df.drop(columns="Goalploeg")
df = df.drop(columns="club_id")
df = df.drop(columns="club_id_y")
df = df.drop(columns="Thuisploeg")
df = df.drop(columns="Uitploeg")
df = df.drop(columns="club_id_x")

In [ ]:
df.info()

In [ ]:
desired_order = ['Seizeon', 'Speeldag', 'Datum', 'Startuur', 'Match_ID', 'huisstamnummer', 'huisroepnaam', 'uitstamnummer', 'uitroepnaam',
'Minuut goal', 'Tijdstip goal', 'goalstamnummer', 'goalroepnaam', 'Thuisstand', "Uitstand"]

df = df[desired_order]

In [ ]:
df.to_csv('doelpunten_fixed.csv', index=False, header=None)